In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob
from pprint import pprint
import json 

## 1. 다양한 문서 형식 처리하기

- RAG (Retrieval Augmented Generation) 시스템의 성능은 뭐니 뭐니 해도 **입력 데이터의 품질**에 크게 좌우됨.
- LangChain은 이런 다양한 형식의 문서들을 일관된 `Document` 객체로 변환해주는 **'Document Loaders'** 기능을 제공함. 
- `Document` 객체는 기본적으로 두 가지 핵심 정보를 가짐:
    - `page_content` (문자열): 문서의 실제 텍스트 내용임. LLM이 이해하고 답변을 생성하는 기반이 됨.
    - `metadata` (딕셔너리): 문서의 출처, 파일명, 페이지 번호, 생성 날짜 등 부가 정보를 담음. 답변의 출처를 표시하거나 특정 조건의 문서를 필터링할 때 아주 유용함.

### 1.1 PDF 문서 (`PyPDFLoader`)

- `PyPDFLoader`는 PDF 파일에서 텍스트를 쪽쪽 빨아들여 추출함.
- 기본적으로 PDF의 각 페이지를 별도의 `Document` 객체로 만듦. 페이지 구분이 명확해서 좋음.
- PDF 파일 내부에 텍스트 레이어가 잘 살아있을수록 추출 품질이 좋음. 눈으로 보이는 거랑 실제 데이터랑 다를 수 있음.
- 이미지로만 이루어진 PDF (예: 스캔본)의 경우, `PyPDFLoader`는 텍스트를 추출하지 못함. 이럴 땐 OCR(광학 문자 인식) 처리가 가능한 다른 로더나 별도 OCR 과정이 필요함.

**장점:**
- 세상에서 가장 많이 쓰이는 문서 형식 중 하나라 활용도가 높음.
- 페이지 번호 같은 메타데이터가 자동으로 추출되어, 나중에 답변 출처를 밝힐 때 매우 유용함 (예: "이 내용은 문서 3페이지에서 가져왔습니다.").
- 비교적 사용법이 간단함.

**단점:**
- 표, 복잡한 레이아웃, 이미지 설명 등이 많은 PDF는 텍스트 추출이 완벽하지 않을 수 있음. 순서가 꼬이거나 일부 누락될 수 있음.
- 스캔된 이미지로만 구성된 PDF는 텍스트 추출 불가능. (이땐 `UnstructuredPDFLoader`에 OCR 옵션을 주거나, `pytesseract` 같은 OCR 라이브러리로 전처리 필요)
- 암호화된 PDF는 당연히 로드 못함. (암호 해제 필요)

**꿀팁 & 노하우:**
- **대안 로더 사용 고려:** `PyPDFLoader`가 시원찮을 땐 `UnstructuredPDFLoader` (OCR 기능 내장 가능), `PyPDFium2Loader` (속도나 특정 PDF 구조에서 유리할 수 있음), `PDFMinerLoader` 등 다른 PDF 로더를 써보는 것도 방법임. 각 로더마다 장단점이 있으니 상황에 맞게 선택!
- **OCR의 중요성:** 스캔된 문서가 많다면, 처음부터 OCR 기능이 있는 로더를 쓰거나, `Google Cloud Vision API`, `Azure Computer Vision` 같은 전문 OCR 서비스를 연동하는 걸 고려해볼 것. 품질이 중요하니까!
- **전처리/후처리:** 로드된 `page_content`가 지저분하다면, 정규표현식이나 텍스트 클리닝 라이브러리를 사용해 불필요한 공백, 머리글/바닥글 등을 제거하는 후처리 작업이 RAG 성능에 도움될 수 있음.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('../data/transformer.pdf')
pdf_docs = pdf_loader.load() # Document 객체의 리스트로 반환됨

print(f"PDF에서 로드된 Document 수 (페이지 수와 같을 가능성 높음): {len(pdf_docs)}")
print(f"첫 번째 페이지 메타데이터: {pdf_docs[0].metadata}")
print(f"첫 번째 페이지 내용 일부: \n{pdf_docs[0].page_content[:500]}")

PDF에서 로드된 Document 수 (페이지 수와 같을 가능성 높음): 15
첫 번째 페이지 메타데이터: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/transformer.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}
첫 번째 페이지 내용 일부: 
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kais

### 1.2 웹 문서 (`WebBaseLoader`)
- `WebBaseLoader`는 지정된 URL의 웹 페이지 내용을 긁어옴.
- HTML 태그를 파싱해서 사람이 읽을 수 있는 텍스트 콘텐츠 위주로 추출함.

**장점:**
- 인터넷상의 방대한 최신 정보를 실시간으로 가져와 RAG 시스템에 활용 가능함. 이게 진짜 강력함.
- 여러 URL을 리스트로 넘겨 한 번에 여러 페이지 내용을 긁어올 수 있음.
- 메타데이터로 `source` (URL), `title` (웹페이지 제목) 등이 자동으로 잡히는 경우가 많음.

**단점:**
- JavaScript로 동적으로 렌더링되는 콘텐츠(SPA, Single Page Application 같은 사이트)는 제대로 못 가져올 수 있음. 껍데기만 가져오는 수가 있음.
- <U>광고, 메뉴, 사이드바, 푸터 등 실제 내용과 관련 없는 잡다한 텍스트가 함께 추출될 수 있어, LLM에 혼란을 줄 수 있음.</U> 후처리가 중요함.
- 웹사이트 구조에 따라 추출 품질이 천차만별임.
- 너무 많은 요청을 짧은 시간에 보내면 사이트에서 차단당할 수 있음. (Rate limit 조심)

**꿀팁 & 노하우:**
- **동적 콘텐츠 로딩:** JS 렌더링이 필요한 사이트는 `WebBaseLoader` 대신 `PlaywrightURLLoader`나 `SeleniumURLLoader`를 사용해야 함. 이들은 실제 웹 브라우저를 백그라운드에서 돌려서 페이지를 가져오므로 동적 콘텐츠도 잘 가져옴. (대신 좀 느리고, playwright/selenium 라이브러리 설치 필요)
- **선택적 추출 (CSS 선택자/XPath):** `BeautifulSoupWebScraper`나 `RecursiveUrlLoader`와 함께 `bs_kwargs` 같은 옵션을 활용하면 특정 HTML 태그 (예: `<article>`, `<main>`) 안의 내용만 선택적으로 가져올 수 있음. 훨씬 깔끔한 데이터를 얻을 수 있음.
- **후처리:** 로드 후 `BeautifulSoup` 라이브러리를 직접 써서 불필요한 태그 ( `<nav>`, `<footer>`, `<script>`)를 제거하거나, 정규표현식으로 광고 문구 등을 필터링하는 후처리 로직을 추가하면 좋음.
- **User-Agent 설정:** 일부 웹사이트는 자동화된 요청을 막기 위해 User-Agent를 확인하기도 함. `WebBaseLoader` 생성 시 `header_template` 이나 유사 옵션을 통해 브라우저처럼 보이도록 User-Agent를 설정해주면 도움이 될 수 있음.
- **Sitemap 활용:** 웹사이트의 sitemap.xml을 분석하면 해당 사이트의 주요 페이지 URL 목록을 쉽게 얻을 수 있음. 이를 `WebBaseLoader`에 전달하면 효율적으로 사이트 전체 정보를 긁어올 수 있음.

In [4]:
from langchain_community.document_loaders import WebBaseLoader

# 여러 웹 페이지 URL을 리스트로 전달하여 로드합니다.
web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])

web_docs = web_loader.load()

print(f"웹에서 로드된 Document 수: {len(web_docs)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


웹에서 로드된 Document 수: 2


In [5]:
# 첫 번째 웹 Document의 메타데이터 확인
# 'source'는 URL, 'title', 'description', 'language' 등 웹페이지 메타 정보가 포함될 수 있습니다.
web_docs[0].metadata

{'source': 'https://python.langchain.com/',
 'title': 'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain',
 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).',
 'language': 'en'}

In [6]:
# 첫 번째 웹 Document의 페이지 내용 (일부만 출력)
print(web_docs[0].page_content[:500])






Introduction | ü¶úÔ∏èüîó LangChain






Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1üí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augment


### 1.3 JSON 파일 (`JSONLoader`)

- `JSONLoader`는 JSON 파일이나 JSONL (JSON Lines) 파일에서 데이터를 로드함.
- **`jq_schema`** 파라미터가 핵심임! JSON 구조 내 특정 경로의 데이터만 쏙쏙 뽑아올 수 있게 해줌. `jq`는 JSON 데이터를 다루는 강력한 커맨드라인 도구인데, LangChain이 이 `jq`의 문법을 활용함.

**`jq_schema` 간단 설명:**
- `.` : 현재 JSON 객체 또는 배열 전체를 의미함.
- `.key` : 현재 객체에서 'key'에 해당하는 값을 선택함.
- `.array[]` : 'array'라는 키를 가진 배열의 모든 요소를 하나하나 선택함 (그래서 여러 `Document`가 생성됨).
- `.array[].field` : 'array' 배열의 각 요소(객체)가 있다면, 그 객체 안의 'field' 키 값을 선택함.

**`text_content=True` vs `text_content=False` (중요!):**
- `text_content=True` (기본값): `jq_schema`로 선택된 값이 문자열이 아니면 (예: 숫자, 리스트, 객체), 이걸 **문자열로 변환**해서 `page_content`에 저장함. 선택된 값이 원래 문자열이면 그대로 사용.
- `text_content=False`: `jq_schema`로 선택된 값을 **JSON 문자열 형태로** `page_content`에 저장함. 즉, 객체나 배열이 선택되면 그 구조가 그대로 문자열화 되어 저장됨. (예: `{"name": "홍길동", "age": 30}` 이런 식으로)

**`content_key` (JSONL 또는 `jq_schema`가 객체를 반환할 때 유용):**
- `jq_schema`가 객체(딕셔너리)를 가리키도록 설정했을 때, 그 객체 안의 특정 필드 값을 `page_content`로 사용하고 싶을 때 지정함. (예: `jq_schema=".messages[]"`, `content_key="text"` 라면, messages 배열 각 요소 객체에서 "text" 필드의 값을 `page_content`로 가져옴)

**`json_lines=True` (JSONL 파일 처리):**
- 파일이 JSONL 형식(한 줄에 하나의 독립적인 JSON 객체가 있는 형태)일 때 이 옵션을 `True`로 설정해야 함.

**장점:**
- 정형화된 데이터를 유연하게 처리 가능함. 특히 API 응답이나 로그 데이터 처리에 좋음.
- `jq_schema`를 통해 필요한 정보만 정확히 추출해서 `page_content`로 만들 수 있음. 데이터 정제가 수월해짐.
- `metadata_func`를 사용하면 원본 JSON 레코드에서 원하는 정보를 뽑아 `metadata`에 커스텀하게 추가할 수 있음. (예: 작성자, 생성 시간 등)

**단점:**
- `jq_schema` 문법에 대한 이해가 필요함. 처음엔 좀 헷갈릴 수 있음.
- 아주 복잡하게 중첩된 JSON 구조의 경우 `jq_schema`가 길어지거나 복잡해질 수 있음.
- `text_content=False` 사용 시, 유니코드 문자(특히 한글)가 `\uXXXX` 형태로 이스케이프 되어 나올 수 있어 후처리가 필요할 수 있음.

**꿀팁 & 노하우:**
- **`jq` 연습:** `jq` 문법이 낯설다면, [jq play](https://jqplay.org/) 같은 온라인 사이트나 로컬에 `jq`를 설치해서 간단한 JSON 파일로 연습해보는 것이 좋음.
- **`metadata_func` 활용:** `metadata_func`를 정의해서 각 JSON 레코드의 중요 필드(날짜, 사용자 ID, 카테고리 등)를 `Document`의 메타데이터로 뽑아내면, 나중에 검색 결과를 필터링하거나 정렬할 때 매우 유용함.
- **한글 깨짐 방지 (유니코드 이스케이프 처리):** `text_content=False`로 객체를 문자열화했을 때 한글이 `\uXXXX` 형태로 나온다면, 로드 후 `doc.page_content = json.dumps(json.loads(doc.page_content), ensure_ascii=False)` 와 같이 처리해주면 사람이 읽을 수 있는 한글로 바뀜.
- **`content_key`와 `jq_schema` 조합:** `jq_schema`로 원하는 객체(들)를 선택하고, `content_key`로 그 객체 안의 특정 텍스트 필드를 `page_content`로 지정하는 패턴이 매우 자주 쓰임.

In [7]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="../data/kakao_chat.json",
    jq_schema=".messages[].content",    # messages 배열의 각 요소(객체)에서 'content' 필드 값을 추출
    text_content=True,                  # 추출된 content 필드가 텍스트이므로 True (사실상 문자열이면 True/False 큰 차이 없음)
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata) # 기본적인 파일 정보만 포함
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content) # 'content' 필드의 텍스트만 추출됨

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': 'D:\\git\\langchain\\src\\data\\kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [8]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="../data/kakao_chat.json",
    jq_schema=".messages[]",    # messages 배열의 각 객체 전체를 추출
    text_content=False,          # 객체를 JSON 문자열 형태로 page_content에 저장
)

json_docs_obj_as_str = json_loader.load()

print("문서의 수:", len(json_docs_obj_as_str))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs_obj_as_str[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs_obj_as_str[0].page_content) # 객체가 문자열화 되어 저장, 유니코드 이스케이프 주의

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': 'D:\\git\\langchain\\src\\data\\kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 {"sender": "\uae40\ucca0\uc218", "timestamp": "2023-09-15 09:30:22", "content": "\uc548\ub155\ud558\uc138\uc694 \uc5ec\ub7ec\ubd84, \uc624\ub298 \ud68c\uc758 \uc2dc\uac04 \ud655\uc778\ucc28 \uc5f0\ub77d\ub4dc\ub9bd\ub2c8\ub2e4."}


**유니코드 디코딩 및 `Document` 객체 재구성 (꿀팁 적용!)**
- 위 예제처럼 `text_content=False`로 객체를 문자열화하면 한글이 `\uXXXX` (유니코드 이스케이프) 형태로 나올 수 있음.
- 이걸 사람이 읽기 좋게, 그리고 LLM이 잘 이해하도록 하려면 다시 파싱해서 예쁜 한글로 만들어줘야 함.
- `json.loads()`로 JSON 문자열을 파이썬 딕셔너리로 변환하고, `json.dumps(..., ensure_ascii=False)`를 사용하면 한글이 깨지지 않는 JSON 문자열로 만들 수 있음. 이 문자열을 새 `page_content`로 사용하거나, 여기서 필요한 정보만 뽑아 `page_content`를 재구성할 수 있음.

In [9]:
from langchain_core.documents import Document 
import json

decoded_json_docs = []
for doc in json_docs_obj_as_str: 
    # page_content (JSON 문자열)를 파이썬 딕셔너리로 변환
    decoded_data = json.loads(doc.page_content)

    # 파이썬 딕셔너리를 한글이 유지되는 "예쁜" JSON 문자열로 변환하여 새로운 Document 객체 생성
    # 또는, decoded_data에서 특정 필드만 뽑아 page_content를 구성할 수도 있음.
    # 예를 들어, f"발신자: {decoded_data['sender']}\n내용: {decoded_data['content']}" 처럼.
    pretty_json_string = json.dumps(decoded_data, ensure_ascii=False, indent=2) # indent로 가독성도 높임
    
    document_obj = Document(
        page_content=pretty_json_string, # 예쁘게 만든 JSON 문자열을 내용으로
        metadata=doc.metadata # 기존 메타데이터는 그대로 유지
    )
    decoded_json_docs.append(document_obj)

print("문서의 수:", len(decoded_json_docs))
print("-" * 50)
print("첫 번째 문서의 메타데이터: \n", decoded_json_docs[0].metadata)
print("-" * 50)
print("첫 번째 문서의 내용 (디코딩 및 예쁘게 재구성 후): \n", decoded_json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
첫 번째 문서의 메타데이터: 
 {'source': 'D:\\git\\langchain\\src\\data\\kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
첫 번째 문서의 내용 (디코딩 및 예쁘게 재구성 후): 
 {
  "sender": "김철수",
  "timestamp": "2023-09-15 09:30:22",
  "content": "안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다."
}


**`metadata_func`를 이용한 커스텀 메타데이터 추출**
- `JSONLoader`의 `metadata_func` 파라미터에 함수를 전달하면, 원본 JSON 레코드(객체)에서 원하는 정보를 추출하여 `Document` 객체의 `metadata`에 추가할 수 있음. 
- RAG 시 검색된 청크의 출처(예: 작성자, 날짜, 카테고리)나 추가 정보를 제공하는 데 아주 유용함. LLM이 답변할 때 이 메타데이터를 참고할 수도 있고, 사용자에게 보여줄 수도 있음.

**`content_key` 다시 한번!**
- `jq_schema`가 리스트의 각 **객체**를 가리키도록 설정했을 때 (예: `.messages[]`), 그 객체 내부의 어떤 **필드 값**을 `page_content`로 사용할지 지정하는 것이 `content_key`임.
- 만약 `content_key`를 지정하지 않으면, `jq_schema`로 선택된 객체 전체가 (문자열화되어) `page_content`가 됨 (위의 `text_content=False` 예제처럼).

In [10]:
# 메타데이터를 추출하는 함수 정의
def chat_metadata_func(record: dict, metadata: dict) -> dict:
    # record: jq_schema에 의해 선택된 JSON 객체 (여기서는 .messages[]의 각 대화 메시지 객체)
    # metadata: 기본적으로 생성되는 메타데이터 (source, seq_num 등)
    
    metadata["sender"] = record.get("sender")       # record에서 'sender' 값을 가져와 메타데이터에 추가
    metadata["timestamp"] = record.get("timestamp") # record에서 'timestamp' 값을 가져와 메타데이터에 추가
    # record에 없는 키를 .get()으로 접근하면 에러 대신 None을 반환해서 안전함
    
    # 기존 metadata에 source, seq_num이 이미 있으므로, chatroom_name은 최상위 객체에서 가져와야 함.
    # 하지만 metadata_func는 각 record 단위로 호출되므로, 여기서는 record 내 필드만 접근하는게 일반적.
    # 만약 파일 전체 정보(chatroom_name)를 넣고 싶다면, 로더 외부에서 파일을 한번 읽어서 처리하거나,
    # 로드 후 모든 Document의 metadata에 일괄 추가하는 방식이 필요.
    
    return metadata


json_loader_with_meta_content_key = JSONLoader(
    file_path="../data/kakao_chat.json",
    jq_schema=".messages[]",  # messages 배열의 각 객체를 대상으로 함
    content_key="content",    # 각 메시지 객체에서 'content' 필드 값을 page_content로 사용
    metadata_func=chat_metadata_func, # 위에서 정의한 함수를 메타데이터 추출에 사용
    text_content=True           # content_key로 지정된 값이 텍스트이므로 True가 자연스러움
)

json_docs_with_meta_content_key = json_loader_with_meta_content_key.load()

print("문서의 수:", len(json_docs_with_meta_content_key))
print("-" * 50)
print("첫 번째 문서의 메타데이터 (커스텀 추가됨): \n", json_docs_with_meta_content_key[0].metadata)
print("-" * 50)
print("첫 번째 문서의 내용: \n", json_docs_with_meta_content_key[0].page_content)

문서의 수: 5
--------------------------------------------------
첫 번째 문서의 메타데이터 (커스텀 추가됨): 
 {'source': 'D:\\git\\langchain\\src\\data\\kakao_chat.json', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}
--------------------------------------------------
첫 번째 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


**JSONL (JSON Lines) 파일 처리 (`json_lines=True`)**

JSONL은 각 줄(line)이 독립적인 JSON 객체인 텍스트 파일 형식임. 로그 데이터나 대량의 JSON 객체를 스트리밍 방식으로 처리할 때 유용함. `JSONLoader`에서 `json_lines=True`로 설정하고, `jq_schema="."` (각 줄 전체를 의미)로 하면 각 줄의 JSON 객체를 하나의 `Document`로 로드할 수 있음. 이때 `content_key`나 `metadata_func`도 동일하게 사용 가능함.

In [11]:
# kakao_chat.jsonl 파일은 각 줄이 아래와 같은 JSON 객체로 되어 있다고 가정함:
# {"sender": "김철수", "timestamp": "2023-09-15 09:30:22", "content": "안녕하세요..."}
# {"sender": "이영희", "timestamp": "2023-09-15 09:31:05", "content": "네, 안녕하세요..."}

jsonl_loader_content_key = JSONLoader(
    file_path="../data/kakao_chat.jsonl", # 이 파일이 실제 있어야 함!
    jq_schema=".",          # 각 줄(JSON 객체) 전체를 대상으로 함
    content_key="content",  # 각 줄(객체)에서 'content' 필드 값을 page_content로 사용
    json_lines=True,      # JSONL 형식임을 명시!
    text_content=True
)

jsonl_docs_content_key = jsonl_loader_content_key.load()

if jsonl_docs_content_key:
    print("문서의 수:", len(jsonl_docs_content_key))
    print("-" * 50)
    print("첫 번째 문서의 메타데이터 (커스텀 없음, 기본만): \n", jsonl_docs_content_key[0].metadata) # file_path, seq_num 등 기본 메타데이터
    print("-" * 50)
    print("첫 번째 문서의 내용: \n", jsonl_docs_content_key[0].page_content)
else:
    print("JSONL 파일에서 문서를 로드하지 못했거나 파일이 비어있음.")

문서의 수: 5
--------------------------------------------------
첫 번째 문서의 메타데이터 (커스텀 없음, 기본만): 
 {'source': 'D:\\git\\langchain\\src\\data\\kakao_chat.jsonl', 'seq_num': 1}
--------------------------------------------------
첫 번째 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [12]:
jsonl_loader_with_meta = JSONLoader(
    file_path="../data/kakao_chat.jsonl",
    jq_schema=".",                 # 각 줄(JSON 객체) 전체를 대상으로 함
    content_key="content",         # 객체에서 'content' 필드 값을 page_content로 사용
    metadata_func=chat_metadata_func,   # 위에서 정의한 함수를 메타데이터 추출에 사용
    json_lines=True,             # JSONL 형식임을 명시
    text_content=True
)

jsonl_docs_with_meta = jsonl_loader_with_meta.load()

if jsonl_docs_with_meta:
    print("문서의 수:", len(jsonl_docs_with_meta))
    print("-" * 50)
    print("첫 번째 문서의 메타데이터 (커스텀 추가됨): \n", jsonl_docs_with_meta[0].metadata)
    print("-" * 50)
    print("첫 번째 문서의 내용: \n", jsonl_docs_with_meta[0].page_content)
    
    # json_docs 변수를 이후 정규표현식 분할 예제 등에서 사용하기 위해 jsonl_docs_with_meta로 할당 (예시)
    # 여기서는 마지막 JSON 관련 로드 결과인 jsonl_docs_with_meta를 json_docs로 통일성 있게 사용하기 위함.
    json_docs = jsonl_docs_with_meta 
else:
    print("JSONL 파일(메타함수)에서 문서를 로드하지 못했거나 파일이 비어있음.")
    json_docs = [] # 이후 에러 방지를 위해 빈 리스트로 초기화

문서의 수: 5
--------------------------------------------------
첫 번째 문서의 메타데이터 (커스텀 추가됨): 
 {'source': 'D:\\git\\langchain\\src\\data\\kakao_chat.jsonl', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}
--------------------------------------------------
첫 번째 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


### 1.4 CSV 문서 (`CSVLoader`)

- `CSVLoader`는 CSV (Comma-Separated Values) 파일을 로드함. 엑셀에서 흔히 보는 그 형식 맞음.
- 기본적으로 CSV 파일의 **각 행(row)을 하나의 `Document` 객체**로 만듦.
- 각 `Document`의 `page_content`는 해당 행의 **모든 열(column) 내용을 "컬럼명1: 값1, 컬럼명2: 값2, ..." 형태의 문자열로 조합**하여 생성됨. (이 방식이 맘에 안들면 후처리가 필요할 수 있음)

**주요 파라미터:**
- `file_path`: CSV 파일 경로.
- `source_column` (선택 사항): 특정 열의 값을 각 `Document`의 `metadata` 안에 `source` 키의 값으로 사용하도록 지정할 수 있음. 예를 들어, CSV에 문서 ID나 제목이 담긴 열이 있다면 그걸 `source`로 지정하면 좋음. 지정 안 하면 파일 경로가 `source`가 됨.
- `csv_args` (선택 사항): CSV 파싱과 관련된 추가 인자들을 딕셔너리 형태로 전달할 수 있음. 예를 들어 구분자(delimiter)가 콤마(,)가 아닌 탭(\t)이나 세미콜론(;)인 경우, 또는 특정 문자를 quotechar로 사용하는 경우 등을 지정. (예: `csv_args={'delimiter': '\t', 'quotechar': '"'}`)
- `encoding` (중요!): 파일 인코딩을 지정함. 특히 **한글이 포함된 CSV 파일**을 다룰 때는 `'utf-8'`, `'cp949'`, `'euc-kr'` 등을 정확히 지정해줘야 글자가 깨지지 않음. (엑셀에서 저장한 CSV는 `'cp949'`인 경우가 많음)

**장점:**
- 표(tabular) 형식의 데이터를 RAG 시스템에 쉽게 통합할 수 있음.
- 각 행이 개별적인 정보 단위를 나타낼 때 (예: 상품 목록, 고객 정보, FAQ 리스트 등) 유용함.
- `source_column`을 사용하면 각 행(문서)의 출처를 명확히 할 수 있음.

**단점:**
- `page_content`가 기본적으로 모든 열의 내용을 합친 문자열이라, 특정 열의 긴 텍스트나 복잡한 구조의 데이터를 표현하기에는 부적합할 수 있음. LLM이 정보를 파악하기 어려울 수도 있음.
- 특정 열(들)만 `page_content`로 사용하고 싶다면, `CSVLoader` 자체에는 직접적인 옵션이 없을 수 있어 로드 후 추가 처리가 필요함 (아래 팁 참고).
- 큰 CSV 파일의 경우 모든 행을 메모리에 로드하므로 메모리 사용량에 주의해야 함.

**꿀팁 & 노하우:**
- **`encoding` 확인 필수:** 한글 CSV 파일 로드 시 글자가 깨진다면 십중팔구 `encoding` 문제임. 파일 편집기(VSCode 등)에서 파일 인코딩을 확인하고 `CSVLoader`에 정확히 알려줄 것.
- **`source_column` 적극 활용:** 데이터에 행을 식별할 수 있는 ID나 제목 컬럼이 있다면 `source_column`으로 지정해서 메타데이터에 넣어주면 좋음. 나중에 정보 추적에 용이함.
- **특정 열만 `page_content`로 사용하기:**
    1. **로드 후 재가공:** `CSVLoader`로 일단 다 읽은 다음, `Document` 리스트를 순회하면서 각 `Document`의 `page_content`를 원하는 열의 내용으로 직접 재구성하는 방법. (예: `doc.page_content = f"질문: {doc.metadata['질문컬럼명']}\n답변: {doc.metadata['답변컬럼명']}"` 이런 식으로. 단, 이러려면 모든 컬럼이 메타데이터에 자동으로 들어가도록 `CSVLoader`가 동작해야 하는데, 확인 필요. `CSVLoader`는 기본적으로 모든 열을 `page_content`에 넣고, `metadata`에는 `row` 번호와 `source`만 넣을 수 있음. 따라서 이 방법보다는 아래 Pandas 활용이 더 현실적임.)
    2. **Pandas 활용 (강력 추천):** `pandas` 라이브러리로 CSV를 먼저 DataFrame으로 읽어온 다음, 원하는 열들만 선택하고 조합해서 `page_content`를 만들고, 다른 필요한 열들은 `metadata`로 구성하여 LangChain `Document` 객체 리스트를 직접 만드는 것이 훨씬 유연하고 강력함. (예: `docs = [Document(page_content=row['답변'], metadata={'질문': row['질문'], '출처': row['출처']}) for index, row in df.iterrows()]`)
- **`csv_args`로 파싱 옵션 조절:** 파일이 표준 CSV 형식이 아닐 때 (예: 구분자가 다르거나, 특정 방식으로 따옴표 처리) `csv_args`를 잘 활용하면 대부분의 CSV 파일을 읽을 수 있음. `python` 내장 `csv` 모듈의 `reader`나 `DictReader`가 받는 인자들을 참고하면 됨.

In [13]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# kbo_teams_2023.csv 파일 사용
csv_loader = CSVLoader(
    "../data/kbo_teams_2023.csv", 
    encoding='utf-8', # 이 CSV 파일은 utf-8로 저장되어 있다고 가정
    # source_column="Team" # 'Team' 열을 메타데이터의 source로 사용하고 싶다면 주석 해제
)
csv_docs = csv_loader.load()

if csv_docs:
    print("문서의 수 (CSV 행의 수와 같음):", len(csv_docs))
    print("-" * 50)
    print("첫 번째 문서의 메타데이터: \n", csv_docs[0].metadata) # 'source'는 파일 경로 (또는 source_column 값), 'row'는 행 번호 (0부터 시작)
    print("-" * 50)
    print("첫 번째 문서의 내용 (모든 컬럼이 조합됨): \n", csv_docs[0].page_content)
    print("-" * 50)
    # 만약 source_column="Team"으로 지정했다면 아래처럼 확인 가능
    # print(f"Team 컬럼이 source로 지정된 경우 첫번째 문서의 source: {csv_docs[0].metadata.get('source')}")
else:
    print("CSV 파일에서 문서를 로드하지 못했거나 파일이 비어있음.")

문서의 수 (CSV 행의 수와 같음): 10
--------------------------------------------------
첫 번째 문서의 메타데이터: 
 {'source': '../data/kbo_teams_2023.csv', 'row': 0}
--------------------------------------------------
첫 번째 문서의 내용 (모든 컬럼이 조합됨): 
 Team: KIA 타이거즈
City: 광주
Founded: 1982
Home Stadium: 광주-기아 챔피언스 필드
Championships: 11
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.
--------------------------------------------------
